두번째 공부하는 Saturn Voyager

운석 무리를 피해서 어디까지 진행하는지를 겨루는 게임입니다. 언뜻 3D처럼 보일지도 모르겠지만 모든 이미지는 2D 이미지를 사용하고 있습니다. 

In [9]:
# saturn voyger second try  
import sys
import pygame
import time
from random import randint
from pygame.locals import QUIT,KEYDOWN,KEYUP,K_LEFT,K_RIGHT,K_SPACE,K_UP,K_DOWN


pygame.init()
SURFACE = pygame.display.set_mode((800,800))
FPSCLOCK = pygame.time.Clock()
pygame.mixer.music.load(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\1_sounds\Escape.mp3")
pygame.mixer.music.play()

hit_sound = pygame.mixer.Sound(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\1_sounds\bang.wav")   # 짧은 wav/mp3 파일
# bang_sound = pygame.mixer.Sound("bang.wav")   # 짧은 wav/mp3 파일
# eat_sound = pygame.mixer.Sound("eat.wav")   # 짧은 wav/mp3 파일
# bomb_sound = pygame.mixer.Sound("bomb.mp3")   # 짧은 wav/mp3 파일
# 실행 hit_sound.play()
###시작준비화면####################################################################################
from pygame.locals import KEYDOWN,K_SPACE
pygame.display.set_caption("Saturn Voyger") # 글꼴 설정
font = pygame.font.SysFont(None, 72)
text = font.render("Start press SPACE!", True, (100,255,100))
text_rect = text.get_rect(center=(400,400))

# --- 1단계: 대기 화면 ---
waiting = True
while waiting:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            raise SystemExit
        if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
            waiting = False  # 스페이스키 눌리면 대기 종료
    SURFACE.fill((0,0,0))
    SURFACE.blit(text, text_rect)
    pygame.display.flip()
    FPSCLOCK.tick(30)
###################################################################################

def main():
    game_over = False
    score = 0
    speed =  25
    stars =[]   #운석을 저장하는 리스트
    keymap = [] #어느키가 입력되어있는지를 나타내는 리스트
    ship =[0,0]  #우주선의 현재위치(x,y)
    scope_image = pygame.image.load(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\2_images\scope.png")
    rock_image = pygame.image.load(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\2_images\rock.png")
    scorefont = pygame.font.SysFont(None,36)
    sysfont = pygame.font.SysFont(None,72)
    message_over= sysfont.render("GAME OVER!!",True,(0,255,255))
    message_rect = message_over.get_rect()
    message_rect.center = (400,300)
    
    while len(stars) <200:
        stars.append({"pos":[randint(-1600,1600),randint(-1600,1600),randint(1,4095)],"theta": randint(0,360)})

    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == KEYDOWN:
                if not event.key in keymap:
                    keymap.append(event.key)
            elif event.type == KEYUP:
                if event.key in keymap:
                    keymap.remove(event.key)

        # 1프레임마다 처리
        if not game_over:
            score +=1
            if score % 10 ==0:
                speed +=1

            if K_LEFT in keymap:
                ship[0] -= 30
            elif K_RIGHT in keymap:
                ship[0] += 30
            elif K_UP in keymap:
                ship[1] -= 30
            elif K_DOWN in keymap:
                ship[1] += 30

        ship[0] = max(-800, min(800,ship[0]))
        ship[1] = max(-800, min(800,ship[1]))

        for star in stars:
            star["pos"][2] -= speed
            if star["pos"][2] <=64:
                if abs(star["pos"][0]-ship[0]) <50 and abs(star["pos"][1]-ship[1]) <50:
                    hit_sound.play()
                    game_over = True
                
                star["pos"]=[randint(-1600,1600),randint(-1600,1600),4095]
        #그리기
        SURFACE.fill((0,0,0)) 

        # try:
        stars = sorted(stars,key=lambda x:x["pos"][2],reverse=True)
        for star in stars:
            zpos= star["pos"][2]
            xpos=((star["pos"][0]-ship[0]) <<9) / zpos+400
            ypos=((star["pos"][1]-ship[1]) <<9) / zpos+400
            size =(50 <<9) / zpos
            rotated = pygame.transform.rotozoom(rock_image,star["theta"],size/145)
            SURFACE.blit(rotated,(xpos,ypos))
        #
            if star["pos"][2] <=1000:  #운석이 가까워졌을때 테두리 표시
                w, h = rock_image.get_size()                 # 2) 테두리용 Surface 생성
                outlined = pygame.Surface((w+4, h+4), pygame.SRCALPHA)
                # 3) 빨간 테두리 그리기
                pygame.draw.rect(outlined, (255, 0, 0), (0, 0, w+4, h+4), 3)
                # 4) 가운데 원본 rock 이미지 붙이기
                outlined.blit(rock_image, (2, 2))
                # 5) 회전 & 크기 적용
                rotated = pygame.transform.rotozoom(outlined, star["theta"], size/145)
                SURFACE.blit(rotated, (xpos, ypos))

        SURFACE.blit(scope_image,(0,0))
        if game_over:
            SURFACE.blit(message_over,message_rect)
            pygame.mixer.music.stop()
        #점수나타내기
        score_str = str(score).zfill(6)
        score_image = scorefont.render(score_str,True,(255,255,0))
        SURFACE .blit(score_image,(700,10))
        
        # ship위치표시
        ship_posx= str(ship[0]).zfill(3)  #즉 zfill(6)은 전체 길이를 6자리로 맞추겠다는 뜻이고, 부족한 자리만큼 왼쪽에 0을 채워 넣는 거지.
        ship_posx_image = scorefont.render(ship_posx,True,(0,255,0))
        SURFACE.blit(ship_posx_image,(100,350))

        ship_posy= str(ship[1]).zfill(3)  #즉 zfill(6)은 전체 길이를 6자리로 맞추겠다는 뜻이고, 부족한 자리만큼 왼쪽에 0을 채워 넣는 거지.
        ship_posy_image = scorefont.render(ship_posy,True,(0,255,0))
        SURFACE.blit(ship_posy_image,(380,120))

        pygame.display.update()
        FPSCLOCK.tick(20)
        

if __name__ =='__main__':
    main()



SystemExit: 